# Runs 

<div style="text-align:center">
    <img src="./images/mlflow_run.jpeg" alt="MLFlow Run" />
</div>


A MLflow run is a unit of work in MLflow that represents the execution of a machine learning experiment or a piece of code. It tracks the parameters, metrics, artifacts, and metadata associated with the run. MLflow runs allow you to log and track experiments, compare different runs, and reproduce results. Each run is associated with an experiment and can have multiple tags, parameters, metrics, and artifacts.

## Creating a MLflow Run

### Using start_run

```python
mlflow.start_run()
```

In [3]:
import mlflow 
from mlflow_for_ml_dev.experiments.exp_utils import get_or_create_experiment

## Creating MLflow experiment

In [4]:
# Define the experiment name and tags
experiment_name = "runs-01"
tags = {"project_name":"UNDEFINED", "topic":"run_management"}
experiment = get_or_create_experiment(experiment_name, tags)

In [ ]:
print(f"Experiment name: {experiment.name}")
print(f"Experiment ID: {experiment.experiment_id}")

## Starting an Active Run.

An active MLflow run is a session where MLflow tracks and logs relevant data about the machine learning model training process. This includes logging parameters, metrics, artifacts (like models or plots), and metadata in real time. A run begins when the mlflow.start_run() function is called and remains active until mlflow.end_run() is executed. During this period, any logged information will be associated with the run, which is identified by a unique run ID.

In [ ]:
run = mlflow.start_run()

In [ ]:
# print the type of the run object
print(f"Object Type: '{type(run).__name__}'")

In [ ]:
run.info.to_proto()

In [ ]:
run.data.to_dictionary()

In [9]:
# Simulating a machine Learning run

# Machine learing code here
# ...

# logging some random parameters
mlflow.log_param("param1", 5)
mlflow.log_param("param2", 5)
mlflow.log_param("param3", 5)

# logging some random metrics
mlflow.log_metric("metric1", 15)
mlflow.log_metric("metric2", 52)
mlflow.log_metric("metric3", 35)

In [10]:
# Get the updated run object. This method will return the updated run object
run = mlflow.get_run(run_id=run.info.run_id)

In [ ]:
run.data.to_dictionary()

### !Error: 

Before creating a new active run is necessary to end any previous run.

In [ ]:
#starting a new run without ending the previous one will throw an error
run2 = mlflow.start_run()

In [13]:
# To start a new run, first end the current run with mlflow.end_run().
mlflow.end_run()

In [14]:
#starting a new run
run2 = mlflow.start_run()

In [15]:
mlflow.end_run()

## Providing more run metadata 

In [1]:
import mlflow 
from mlflow_for_ml_dev.experiments.exp_utils import get_or_create_experiment

In [2]:
# Define the experiment name and tags
experiment_name = "runs-01"
tags = {"project_name":"UNDEFINED", "topic":"run_management"}
experiment = get_or_create_experiment(experiment_name, tags)

In [3]:
# creating run providing more info. Such as run name, description and tags.
run_tags = {"tag1": "value1", "tag2": "value2"}

run3 = mlflow.start_run(
    run_name="run_with_tags",
    tags=run_tags,
    description="This is a run with tags"
)

In [4]:
# ending the run
mlflow.end_run()

#### Using with statement


> About `mlflow.start_run()`
> 
> The return value of `mlflow.start_run()` can be used as a context manager within a `with` block. Otherwise, you must call `end_run()` to terminate the current run.

Example:

```python
with mlflow.start_run() as run:
    print("Log metrics and params")
```

In [5]:
# mlflow.end_run() # end the active run to start a new one
with mlflow.start_run(run_name="Run 2", experiment_id=experiment.experiment_id) as run:

    # Your ML code here
    # ...
    
    active_run = mlflow.active_run()
    print(type(active_run))  
    print("Active Run: ", run.info.run_id)
    print("Active Run: ", active_run.info.run_id)
    print("\n \n")


# outside the with block
active_run = mlflow.active_run()
print(type(active_run))    

<class 'mlflow.tracking.fluent.ActiveRun'>
Active Run:  8ec23e31aa7947099f77aafc894fbf21
Active Run:  8ec23e31aa7947099f77aafc894fbf21

 

<class 'NoneType'>


In [6]:
# active_run is None because the run has ended.
active_run == None

True

## Using MlflowClient

In [7]:
# creating a mlflow client
client = mlflow.MlflowClient()

In [8]:
# This will create a run with the specified name and tags
created_run = client.create_run(
    experiment_id=experiment.experiment_id,
    run_name="test_run",
    tags = {
        "tag1": "value1",
        "tag2": "value2"
        }
    )

In [9]:
# Notice that the run is of type mlflow.entities.Run. Before it was of type mlflow.entities.ActiveRun
type(created_run).__name__

'Run'

In [10]:
# since the client creates a Run object, there will be no active run. (ActiveRun is a different object)
run = mlflow.active_run()
type(run)

NoneType

In [11]:
# Loging to MLflow without an active run will create a new run automatically. 
# Some mlflow functions create a run automatically if there is no active run.
mlflow.log_param("param1", 5)
run = mlflow.active_run()
type(run)

mlflow.tracking.fluent.ActiveRun

In [12]:
# Random name created by mlflow
run.info.run_name

'bright-sloth-381'

In [13]:
# End the active run
mlflow.end_run()

In [14]:
# We can resume the run by providing the run_id and then log into it 
with mlflow.start_run(run_id = created_run.info.run_id):
    mlflow.log_param("param1", 5)
    run = mlflow.active_run()
    print(run.info.run_name)

# This will also end the run

test_run


In [15]:
# Creating a second run with the mlflow client
created_run = client.create_run(
    experiment_id=experiment.experiment_id,
    run_name="test_run",
    tags = {
        "tag1": "value1",
        "tag2": "value2"
    }
)

In [16]:
# There are some logging functions that allows to log into a specific run by providing the run_id
mlflow.log_metric(key="metric1", value=5, run_id=created_run.info.run_id)

In [17]:
# Although, not all logging functions have the run_id parameter.
# For example, mlflow.log_param() does not have the run_id parameter
mlflow.log_param("param1", 5)

5

In [18]:
# ending the active run
mlflow.end_run()

In [19]:
# Instead of using the "with" block, we can use the client to terminate the run.
client.set_terminated(run_id=created_run.info.run_id)

## Updating Run Tags

In [3]:
import mlflow 
from mlflow_for_ml_dev.experiments.exp_utils import get_or_create_experiment

In [4]:
# Define the experiment name and tags
experiment_name = "runs-01"
tags = {"project_name":"UNDEFINED", "topic":"run_management"}
experiment = get_or_create_experiment(experiment_name, tags)

In [5]:
# end the active run to start a new one
mlflow.end_run()
with mlflow.start_run(run_name="Run 3", experiment_id=experiment.experiment_id) as run:
    # set a single tag
    mlflow.set_tag("tag3", "value3")
    
    # Set multiple tags as a dictionary 
    mlflow.set_tags({"tag4": "value4", "tag5": "value5"})

## Update run description

In [7]:
# we can update the description of the run 
# In this case we are updating the description of the run with the name "Run 4"
with mlflow.start_run(run_name="Run 4", experiment_id=experiment.experiment_id) as run:
    
    #Update description
    # the tag "mlflow.note.content" is used to store the description of the run
    mlflow.set_tag("mlflow.note.content", "This is a new description")

In [37]:
# including markdown.
with mlflow.start_run(run_name="Run 5", experiment_id=experiment.experiment_id) as run:
    #Update description
    mlflow.set_tag("mlflow.note.content", "# This is a new description")

    mlflow.log_param("param1", 5)
    mlflow.log_metric("metric1", 15)


## Retrieve run information

In [ ]:
run.info.run_name

In [39]:
run = mlflow.get_run(run_id = run.info.run_id)

In [ ]:
run.data.to_dictionary()